# Get data


In [10]:
from pathlib import Path
import re

import pandas as pd


logs = Path("log")
data = []
p_makespan = r"Total elapsed time:\s+([0-9]+\.[0-9]+(e[+-][0-9]+)?)"
p_lvl_header = r"DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST|  Elapsed time"
p_metric = r""


for filename in logs.glob("*.out"):
    _, sub_id, exp_id, rep_id = filename.stem.split("-")
    file_txt = filename.read_text()
    m = re.search(p_makespan, file_txt)
    # TODO remove condition when all data is available.
    if m:
        total_time = m.group(1)

        iterations = []
        for level in re.split(p_lvl_header, file_txt)[1:-1]:
            last_row = re.split(r"\n", level.strip("XX").strip())[-1].split(",")
            iterations.append(int(last_row[1]))
        metric = abs(float(last_row[2]))

        assert len(iterations) == 4
        l1, l2, l3, l4 = iterations

        data.append((sub_id, exp_id, rep_id, total_time, l1, l2, l3, l4, metric))

data
columns = ["subj_id", "exp_id", "rep_id", "makespan", "l1", "l2", "l3", "l4", "metric"]
df = pd.DataFrame(data, columns=columns)
df["makespan"] = df["makespan"].astype(float)

# Plots

In [11]:
import plotly.graph_objects as go
import pandas as pd


figure_dir = Path("figures")
figure_dir.mkdir(exist_ok=True)

In [12]:
def get_dropdown(subjects):
    # Buttons to toggle between subject and global average
    buttons = [
        dict(
            label="Global Average",
            method="update",
            args=[
                {"visible": [subject == "Global Average" for subject in subjects]},
                {"title": "Global Average"},
            ],
        )
    ]

    # Add buttons for each subject
    for subject in df["subj_id"].unique():
        buttons.append(
            dict(
                label=subject,
                method="update",
                args=[
                    {"visible": [subject == s for s in subjects]},
                    {"title": f"{subject} Average"},
                ],
            )
        )
    
    return buttons

In [13]:
df["iter_time"] = df["makespan"] / df[["l1", "l2", "l3", "l4"]].sum(axis=1)

stat_columns = ["makespan", "l1", "l2", "l3", "l4", "metric", "iter_time"]
subj_stats = (
    df.groupby(["subj_id", "exp_id"])[stat_columns].agg(["mean", "std"]).reset_index()
)
global_stats = df.groupby("exp_id")[stat_columns].agg(["mean", "std"]).reset_index()

global_stats["subj_id"] = "Global Average"

stats = pd.concat([subj_stats, global_stats], ignore_index=True)
stats.columns = stats.columns.map("_".join).str.strip("_")

stats["highlighted_label"] = stats["exp_id"].apply(
    lambda x: x[:-1] + "<b style='color:red;'>" + x[-1] + "</b>"
)

stats = (
    (stats.assign(l=stats["exp_id"].str.len()))
    .sort_values(by=["l", "exp_id"])
    .drop(columns=["l"])
)

subjects = stats["subj_id"].unique()

## Makespan


In [14]:
fig = go.Figure()

# Add traces for each subject and global average
for subject in subjects:
    subject_data = stats[stats["subj_id"] == subject]
    fig.add_trace(
        go.Bar(
            x=subject_data["highlighted_label"],
            y=subject_data["makespan_mean"],
            name=subject,
            visible=(subject == "Global Average"),
            error_y=dict(type="data", array=subject_data["makespan_std"]),
        )
    )

buttons = get_dropdown(subjects)

# Update layout with dropdown menu and fixed y-axis range
fig.update_layout(
    updatemenus=[dict(type="dropdown", showactive=True, buttons=buttons)],
    title="Average makespan",
    xaxis_title="Experiment",
    yaxis_title="Makespan (seconds)",
    yaxis=dict(range=[0, stats["makespan_mean"].max() * 1.05]),
    # height=1000,
)

fig.show()
fig.write_html(figure_dir / "makespan.html")
fig.write_image(figure_dir / "makespan.png")

In [15]:
fig = go.Figure()

# Remove SyN experiments
zoomed_stats = stats[~stats["exp_id"].str.len().eq(3)]

# Add traces for each subject and global average
subjects = zoomed_stats["subj_id"].unique()
for subject in subjects:
    subject_data = zoomed_stats[zoomed_stats["subj_id"] == subject]
    fig.add_trace(
        go.Bar(
            x=subject_data["highlighted_label"],
            y=subject_data["makespan_mean"],
            name=subject,
            visible=(subject == "Global Average"),
            error_y=dict(type="data", array=subject_data["makespan_std"]),
        )
    )

buttons = get_dropdown(subjects)

# Update layout with dropdown menu and fixed y-axis range
fig.update_layout(
    updatemenus=[dict(type="dropdown", showactive=True, buttons=buttons)],
    title="Average makespan (w/o SyN)",
    xaxis_title="Experiment",
    yaxis_title="Makespan (seconds)",
    yaxis=dict(range=[0, zoomed_stats["makespan_mean"].max() * 1.05]),
    # height=1000,
)

fig.show()
fig.write_html(figure_dir / "makespan_wo_syn.html")
fig.write_image(figure_dir / "makespan_wo_syn.png")

## Iterations

In [16]:
fig = go.Figure()

# Add traces for each level and each subject
for subject in subjects:
    subject_data = stats[stats["subj_id"] == subject]
    fig.add_trace(
        go.Bar(
            x=subject_data["highlighted_label"],
            y=subject_data["l1_mean"],
            name=f"l1 ({subject})",
            visible=(subject == "Global Average"),
            error_y=dict(type="data", array=subject_data["l1_std"]),
        )
    )
    fig.add_trace(
        go.Bar(
            x=subject_data["highlighted_label"],
            y=subject_data["l2_mean"],
            name=f"l2 ({subject})",
            visible=(subject == "Global Average"),
            error_y=dict(type="data", array=subject_data["l2_std"]),
        )
    )
    fig.add_trace(
        go.Bar(
            x=subject_data["highlighted_label"],
            y=subject_data["l3_mean"],
            name=f"l3 ({subject})",
            visible=(subject == "Global Average"),
            error_y=dict(type="data", array=subject_data["l3_std"]),
        )
    )
    fig.add_trace(
        go.Bar(
            x=subject_data["highlighted_label"],
            y=subject_data["l4_mean"],
            name=f"l4 ({subject})",
            visible=(subject == "Global Average"),
            error_y=dict(type="data", array=subject_data["l4_std"]),
        )
    )

buttons = get_dropdown(subjects)

# Update layout with dropdown menu and stack bars
y_max = (
    stats.groupby("exp_id")[["l1_mean", "l2_mean", "l3_mean", "l4_mean"]]
    .mean()
    .sum(axis=1)
    .max()
)
fig.update_layout(
    updatemenus=[dict(type="dropdown", showactive=True, buttons=buttons)],
    barmode="stack",
    title="Average number of iterations for each level",
    xaxis_title="Experiment ID",
    yaxis_title="Number of Iterations",
    yaxis=dict(
        range=[
            0,
            y_max * 1.05,
        ]
    ),
)

fig.show()
fig.write_html(figure_dir / "iterations.html")
fig.write_image(figure_dir / "iterations.png")

In [17]:
fig = go.Figure()

# Add traces for each subject and global average
for subject in subjects:
    subject_data = stats[stats["subj_id"] == subject]
    fig.add_trace(
        go.Bar(
            x=subject_data["highlighted_label"],
            y=subject_data["iter_time_mean"],
            name=subject,
            visible=(subject == "Global Average"),
            error_y=dict(type="data", array=subject_data["iter_time_std"]),
        )
    )

buttons = get_dropdown(subjects)

# Update layout with dropdown menu and fixed y-axis range
fig.update_layout(
    updatemenus=[dict(type="dropdown", showactive=True, buttons=buttons)],
    title="Average iteration time",
    xaxis_title="Experiment",
    yaxis_title="Iteration time (seconds)",
    yaxis=dict(range=[0, stats["iter_time_mean"].max() * 1.05]),
    # height=1000,
)

fig.show()
fig.write_html(figure_dir / "iter_time.html")
fig.write_image(figure_dir / "iter_time.png")

## Metric Value

In [18]:
fig = go.Figure()

# Add traces for each subject and global average
for subject in subjects:
    subject_data = stats[stats["subj_id"] == subject]
    fig.add_trace(
        go.Bar(
            x=subject_data["highlighted_label"],
            y=subject_data["metric_mean"],
            name=subject,
            visible=(subject == "Global Average"),
            error_y=dict(type="data", array=subject_data["metric_std"]),
        )
    )

buttons = get_dropdown(subjects)

# Update layout with dropdown menu and fixed y-axis range
fig.update_layout(
    updatemenus=[dict(type="dropdown", showactive=True, buttons=buttons)],
    title="Average metric value",
    xaxis_title="Experiment",
    yaxis_title="Metric value",
    yaxis=dict(range=[0, stats["metric_mean"].max()*1.05]),
    # height=1000,
)

fig.show()
fig.write_html(figure_dir / "metrics.html")
fig.write_image(figure_dir / "metrics.png")